In [1]:
from collections import namedtuple
from random import choice,seed,randint
from tqdm.auto import tqdm
import numpy as np
from icecream import ic
import heapq
import math


SEED = 43242
#SEED = 4
#SEED= randint(0, 2**32 - 1)
seed(SEED)
ic(SEED)

ic| SEED: 43242


43242

In [2]:
def unary_fun(a):
    # Define unary operations
    # Note: np.sqrt by itself would return NaN for negative numbers and issue a RuntimeWarning.
    # The lambda below handles negative inputs for sqrt specifically by returning infinity.
    unary_operations = {  #prova a toglie e vedi se funziona
        # ... (other functions like identity, abs, sqrt, etc. are mostly fine or handled) ...
        "x0": lambda x: x,
        "-x0": lambda x: -x,
        "abs(x0)": lambda x: np.abs(x),
        "sqrt(x0)": lambda x: np.sqrt(x) if x >= 0 else 0.0, # Your existing check
        "square(x0)": lambda x: np.square(x),
        "exp(x0)": lambda x: np.exp(x),
        # Handle log domain errors explicitly (return nan or 0.0)
        "log(x0)": lambda x: np.log(x) if x > 0 else 0.0, # Or 0.0
        "log10(x0)": lambda x: np.log10(x) if x > 0 else 0.0, # Or 0.0
        "log2(x0)": lambda x: np.log2(x) if x > 0 else 0.0, # Or 0.0
        # Handle reciprocal domain error (return inf is standard, or choose nan/0.0)
        "reciprocal(x0)": lambda x: np.reciprocal(x.astype(float)) if x != 0 else 0.0, # Or np.nan/0.0

        "sin(x0)": lambda x: np.sin(x),
        "cos(x0)": lambda x: np.cos(x),
        "tan(x0)": lambda x: np.tan(x),

        # Explicit checks for inverse trig
        "arcsin(x0)": lambda x: np.arcsin(x) if -1 <= x <= 1 else 0.0, # Or 0.0
        "arccos(x0)": lambda x: np.arccos(x) if -1 <= x <= 1 else 0.0, # Or 0.0
        "arctan(x0)": lambda x: np.arctan(x), # Domain is all real numbers

        "sinh(x0)": lambda x: np.sinh(x),
        "cosh(x0)": lambda x: np.cosh(x),
        "tanh(x0)": lambda x: np.tanh(x),

        # Explicit checks for inverse hyperbolic
        "arcsinh(x0)": lambda x: np.arcsinh(x), # Domain is all real numbers
        "arccosh(x0)": lambda x: np.arccosh(x) if x >= 1 else 0.0, # Or 0.0
        "arctanh(x0)": lambda x: np.arctanh(x) if -1 < x < 1 else 0.0, # Or 0.0 (or handle +/-1 specifically)

        "ceil(x0)": lambda x: np.ceil(x),
        "floor(x0)": lambda x: np.floor(x),
        "sign(x0)": lambda x: np.sign(x),
        "ignore(x0)": lambda x: 0.0,
        "factorial(x0)": lambda x: np.math.factorial(x) if x >= 0 and x == int(x) else 0.0,
    }

    results_list = []  # Use a list to collect results
    operations=[]

    # Iterate through the operations, apply them, and store results
    for name, func in unary_operations.items():
        try:
            result = func(a)
            results_list.append(result)
        except Exception as e:
            # print(f"Warning: Error applying {name} to input {a}: {e}") # Optional: for debugging
            results_list.append(0.0)  # Fallback to 0.0 as per original code's attempt
                                     # np.nan might be more appropriate for "undefined"
        operations.append(name)

    return np.array(results_list),operations


def safe_power(base, exponent):
    try:
        # Protect against division by zero and non-finite values
        if not np.isfinite(base) or not np.isfinite(exponent):
            return 0.0

        # Prevent domain issues (negative base with fractional exponent)
        if base < 0 and not float(exponent).is_integer():
            return 0.0

        # Protect zero base with negative exponent
        if base == 0 and exponent < 0:
            return 0.0

        with np.errstate(over='ignore', invalid='ignore', divide='ignore'):
            result = np.power(base, exponent)

        # If result is inf or nan, return 0
        if not np.isfinite(result):
            return 0.0

        return result

    except Exception:
        return 0.0



def binary_fun(x0,x1):
    # Define unary operations
    # Note: np.sqrt by itself would return NaN for negative numbers and issue a RuntimeWarning.
    # The lambda below handles negative inputs for sqrt specifically by returning infinity.
    binary_operations = {
    "x0 - x1": lambda a, b: a - b,
    "x0 + x1": lambda a, b: a + b,
    "x0 * x1": lambda a, b: a * b,
    "x0 / x1": lambda a, b: a / b if b != 0 else 0.0,
    "x0 % x1": lambda a, b: np.mod(a, b) if b != 0 else 0.0,
    "x0 // x1": lambda a, b: np.floor_divide(a, b) if b != 0 else 0.0,
    # Protect against:
    #  - negative base with fractional exponent (complex)
    #  - zero base with negative exponent (division by zero)
    "x0 ^ x1": lambda a, b: safe_power(a, b),
    "x0-root-x1": lambda a, b: safe_power(b, 1/a),
    }

    results_list = []  # Use a list to collect results
    operations=[]
    # Iterate through the operations, apply them, and store results
    for name, func in binary_operations.items():
        try:
            result = func(x0,x1)
            results_list.append(result)
        except Exception as e:
            # print(f"Warning: Error applying {name} to input {a}: {e}") # Optional: for debugging
            results_list.append(0.0)  # Fallback to 0.0 as per original code's attempt
                                     # np.nan might be more appropriate for "undefined"
        operations.append(name)

    

    for name, func in binary_operations.items(): #inverting the order of operands
        try:
            result = func(x1,x0)
            results_list.append(result)
        except Exception as e:
            # print(f"Warning: Error applying {name} to input {a}: {e}") # Optional: for debugging
            results_list.append(0.0)  # Fallback to 0.0 as per original code's attempt
                                     # np.nan might be more appropriate for "undefined"
        operations.append("inv "+name)

    return np.array(results_list),operations


In [ ]:
class Node:
    def init

In [ ]:
worst=0

for i in range(2):
    errore=0
    worst=0

    problem = np.load('data/problem_{}.npz'.format(i))
    x = problem['x']
    y = problem['y']
    ic(x.shape)






    index=0










#diff=y[index]-appy_fun(x[0][index],x[1][index])

    for index in range(y.shape[0]):
        #index=2
        x0=x[0][index]
        if x.shape[0]==1:
            x1=x[0][index]
        else:
           
            
            x1=x[1][index]

        y0=y[index]
